In [1]:
! pip install torch
! pip install numpy
! pip install rdkit
! pip install allennlp-light
! pip install zipfile


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from rdkit import Chem
from model import TrainGAN

In [3]:
import zipfile

# Read the CSV file from the zip file
zip_file_path = "filtered_smiles_dataset.zip"
data = []
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    with zip_ref.open('filtered_smiles_dataset.csv') as csv_file:
        for line in csv_file.readlines()[1:]:
            smile = line.decode('utf-8').strip()
            data.append(smile)

# Define a function to read the top hyperparameters from the file
def read_top_hyperparameters(file_path):
    top_params = []
    with open(file_path, "r") as f:
        for line in f:
            score, params = line.split(', Parameters: ')
            params = eval(params.strip())
            top_params.append(params)
    return top_params


In [4]:
# Read the top hyperparameters
top_hyperparameters = read_top_hyperparameters("best_hyperparams.txt")

best_params = top_hyperparameters[0]
hidden_dim = best_params['hidden_dim']
lr = best_params['lr']
dropout = best_params['dropout']
batch_size = best_params['batch_size']

In [5]:
best_params

{'hidden_dim': 256,
 'lr': 0.0002362588471099862,
 'dropout': 0.3,
 'batch_size': 64}

In [6]:
# Create the model using the best hyperparameters
gan_model = TrainGAN(data, hidden_dim=hidden_dim, lr=lr, device="cuda")

In [7]:
# Update dropout rates in the model
gan_model.generator.output_layer[1].p = dropout
gan_model.generator.output_layer[4].p = dropout
gan_model.discriminator.fc[1].p = dropout
gan_model.discriminator.fc[4].p = dropout

gan_model.generator.project._dropout[0].p = dropout
gan_model.generator.project._dropout[1].p = dropout

print(gan_model.generator.output_layer[1].p)
print(gan_model.generator.project._dropout[0].p) 

0.3
0.3


In [8]:
import math

epoch = len(data)/batch_size # 1 epoch cost 10minutes
steps = epoch * 100 # costs 4 days
rounded_steps = math.ceil(steps / 100) * 100
rounded_steps

206300

In [9]:
# Create dataloader
loader = gan_model.create_dataloader(data, batch_size=batch_size, shuffle=True, num_workers=8)

tokens:  ['.', '7', '9', 'B', '-', '1', '8', 's', 'a', '\\', 'o', '5', 'l', ']', 'A', 'g', '%', '2', '6', 'r', '=', 'O', ')', 'L', '4', 'e', '(', 'S', 'P', '/', 'i', '#', '+', 'F', 'n', '@', 'I', '0', 'N', 'H', '3', '[', 'K', 'C', 'c', 'M', '[C@H]', '[C@@H]', '[nH]', '[O-]', '[C@]', '[N+]', '[C@@]', '[S+]', '<eos>', '<sos>']
vocab_size:  57
start:  56
end:  55


In [10]:
gan_model.train_n_steps(loader, max_step=206300, evaluate_every=100, save_every=20630)


7SN]\/ia\-6[C@@H]N
disc_loss: 0.69738722, gen_loss: 1.15852427, valid 1 =  0.02
(c))C24.c1cCPCC5rN\1
disc_loss: 0.23383799, gen_loss: -4.01097965, valid 2 =  0.00
c22c3C
disc_loss: 0.04292502, gen_loss: 0.79418868, valid 3 =  0.03
ccc2C2c2522CnCcccC(nOcc2c
disc_loss: 0.00702529, gen_loss: -0.15746461, valid 4 =  0.14
Cc1IcO)1C4(1C)1c(O2(c)C
disc_loss: 0.01574175, gen_loss: 0.22207528, valid 5 =  0.08
(Cc/n2nc
disc_loss: 0.00391410, gen_loss: 0.00193376, valid 6 =  0.09
C
disc_loss: 0.03003830, gen_loss: -0.04787258, valid 7 =  0.06
ron[C@H]c23nC[N+]11OcCF))cc1O=122n)11cC2([C@@H]cC/61Cc0)l))ccC1N[C@@H]CC2)[C@H]OCO@)c[C@@H]12N2-C
disc_loss: 0.00552681, gen_loss: 0.17580704, valid 8 =  0.04
C3[C@@H]2FNO1NcO3Ca)ccC(O[C@H](C(nc2O
disc_loss: 0.00537377, gen_loss: -0.16845562, valid 9 =  0.07
C22C1Cc[C@@H]C)1[N+]2C%(C1c3
disc_loss: 0.00143724, gen_loss: -0.10243212, valid 10 =  0.05
C4[C@H])N4[C@@H]4N)[C@@H])c[C@H]CLc1-OcN2c41
disc_loss: 0.00341556, gen_loss: 0.12107533, valid 11 =  0.05
cSOn

0.95

In [11]:
gan_model.eval()

TrainGAN(
  (generator): Generator(
    (embedding_layer): Embedding(56, 256)
    (project): FeedForward(
      (_activations): ModuleList(
        (0): LeakyReLU(negative_slope=0.01)
        (1): ELU(alpha=0.1)
      )
      (_linear_layers): ModuleList(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): Linear(in_features=512, out_features=512, bias=True)
      )
      (_dropout): ModuleList(
        (0-1): 2 x Dropout(p=0.3, inplace=False)
      )
    )
    (rnn): LSTMCell(256, 256)
    (output_layer): Sequential(
      (0): LeakyReLU(negative_slope=0.01)
      (1): Dropout(p=0.3, inplace=False)
      (2): Linear(in_features=256, out_features=512, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Dropout(p=0.3, inplace=False)
      (5): Linear(in_features=512, out_features=55, bias=True)
    )
  )
  (discriminator): Discriminator(
    (embedding): Embedding(57, 256, padding_idx=0)
    (rnn): LstmSeq2SeqEncoder(
      (_module): LSTM(256, 256

In [12]:
smiles_list = gan_model.generate_n(50000)

In [13]:
smiles_list

['Cc1ccc(C(=O)CCC(=O)O)cc1C',
 'CCCN1CCN(C(=O)c2ccc(Cl)cc2)C(=O)c2ccccc21',
 'O=C(NCCc1ccccc1)c1ccc(F)cc1',
 'COc1ccccc1C(=O)O.Cl',
 'c1ccc(COc2ccc(C#N)cc2)cc1',
 'O=[N+]([O-])c1ccc(CCn2ccnc2)cc1',
 'COc1ccc(C(=O)NCC(=O)O)cc1',
 'N#Cc1ccc(C(=O)Nc2ccc(Cl)cc2)cc1',
 'O=C(CS)Nc1ccc(Cl)cc1',
 'COc1ccccc1/C=N/N=C(\\c1ccccc1)c1ccccc1',
 'O=C(NCCc1ccc(F)cc1)NO',
 'CCCCCCCCC(=O)NC[C@H](O)[C@H](O)COc1ccccc1',
 'O=C(CC(=O)Nc1ccccc1Cl)NC(=O)Cc1ccccc1',
 'CCC(=O)Oc1ccc(CC(=O)Nc2ccc(Cl)cc2)cc1',
 'Cc1ccc(CCNCc2cccnc2)cc1',
 'N#Cc1ccc(Cn2cncn2)cc1',
 'COc1ccccc1C(=O)Nc1ccc(C(=O)OCC(=O)O)cc1',
 'COc1cccc(C(=O)/C=C/c2ccc(Cl)cc2)c1',
 'CN(CC)CCCNc1ccc(C(=O)NCCCNC(=O)c2cccs2)cc1',
 'COc1ccccc1/C=C(\\C(=N\\O)c1ccccc1OCC(=O)O',
 'CCOc1ccc(CC(=O)NCc2cccc(F)c2)cc1',
 'CC(=O)NC(=O)c1ccccc1NC(=S)Nc1ccncc1',
 'O=P(O)(O)C(O)P(=O)(O)O.Cl',
 'COc1ccccc1NC(=O)c1ccc(C=O)cc1',
 'O=C(COc1ccc(Cl)cc1O)c1ccc(CO)cc1',
 'COc1ccccc1/C=C/C(=N\\NC(N)=S)c1ccccc1',
 'COc1ccc(CC(=O)/C=C/c2ccc(OC)cn2)cc1',
 'COc1ccc(C(=O)COC(=O)

In [14]:
# Specify the output file name
output_file = 'smiles_list.txt'

# Write the SMILES list to a .txt file
with open(output_file, 'w') as file:
    for smiles in smiles_list:
        file.write(smiles + '\n')

print(f"SMILES list saved to '{output_file}'.")

SMILES list saved to 'smiles_list.txt'.


In [15]:
import torch

# Save generator and discriminator models
torch.save(gan_model.generator.state_dict(), f"11generator.pth")
torch.save(gan_model.discriminator.state_dict(), f"11discriminator.pth")

# Save tokenizer state
tokenizer_state = {
    'mapping': gan_model.tokenizer.mapping,
    'inv_mapping': gan_model.tokenizer.inv_mapping,
    'start_token': gan_model.tokenizer.start_token,
    'end_token': gan_model.tokenizer.end_token,
    'vocab_size': gan_model.tokenizer.vocab_size
}
torch.save(tokenizer_state, f"11tokenizer.pth")

# Save generator optimizer state
torch.save(gan_model.generator_optim.state_dict(), f"11generator_optimizer.pth")

# Save discriminator optimizer state
torch.save(gan_model.discriminator_optim.state_dict(), f"11discriminator_optimizer.pth")

print("Model saved successfully.")




Model saved successfully.
